In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.Engine = os.getenv("ENGINE")
ENGINE = os.getenv("ENGINE")
bing_search_url = os.getenv("BING_SEARCH_API_BASE")
bing_search_api_key = os.getenv("BING_SEARCH_API_KEY")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_KEY")
search_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
search_api_version = os.getenv("AZURE_SEARCH_API_VERSION")


In [2]:
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

chat_model = AzureChatOpenAI(azure_deployment=ENGINE)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage, FunctionMessage

text = """
Write SQL query to get latest quater sales from the database
"""

print(chat_model.invoke(text).content)

Without knowing the structure of the database and how the sales data is stored, it is difficult to provide an exact SQL query. However, a general query to retrieve the latest quarter sales could be:

SELECT SUM(sales_amount) as total_sales 
FROM sales_table 
WHERE sales_date >= DATEADD(qq, DATEDIFF(qq, 0, GETDATE()), 0) 
AND sales_date < DATEADD(qq, DATEDIFF(qq, 0, GETDATE())+1, 0)

This query uses the DATEADD and DATEDIFF functions to calculate the start and end dates of the current quarter, and then filters the sales table to only include sales within that date range. The SUM function is used to calculate the total sales amount for the latest quarter.


In [16]:
db_schema = ""
with open("wwidw.txt", "r") as f:
    db_schema = f.read()


In [17]:
from langchain.prompts.chat import ChatPromptTemplate


system_prompt = """
You are a data analyst working for Wide World Importers. 
You are tasked with writing a SQL query to get latest quater sales from the database. 

--- Database Schema ---
{database_schema}
--- End of Database Schema ---

## Example 
Question: Get top 100 sale records

SQL Query:
```
SELECT TOP (100)
FROM 
    [Fact].[Sale]
```

"""

user_prompt = """
Question: {business_question}
"""

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", user_prompt)
])

msg = chat_prompt.format(database_schema=db_schema, business_question="Get latest quater sales")
print(chat_model.invoke(msg).content)

SQL Query:
```
SELECT 
    *
FROM 
    [Fact].[Sale]
WHERE 
    [Invoice Date Key] >= DATEADD(qq, DATEDIFF(qq, 0, GETDATE()), 0)
```


In [6]:
from langchain.schema import BaseOutputParser

class SQLQueryParser(BaseOutputParser):
    def parse(self, text):
        return text.split("```")[1]
    
print(SQLQueryParser().parse(chat_model.invoke(msg).content))


SELECT 
    [City Key],
    [Customer Key],
    [Stock Item Key],
    [Invoice Date Key],
    [Quantity],
    [Unit Price],
    [Total Excluding Tax],
    [Tax Amount],
    [Total Including Tax],
    [Profit]
FROM 
    [Fact].[Sale]
WHERE
    [Invoice Date Key] >= DATEADD(qq, DATEDIFF(qq, 0, GETDATE())-1, 0) AND
    [Invoice Date Key] < DATEADD(qq, DATEDIFF(qq, 0, GETDATE()), 0)



In [18]:
chain = chat_prompt | chat_model | SQLQueryParser()
sql_query = chain.invoke({"database_schema": db_schema, "business_question": "Get latest quater sales"})
print(sql_query)


SELECT 
    [Stock Item],
    [City],
    [Quantity],
    [Unit Price],
    [Total Including Tax]
FROM 
    [Fact].[Sale]
    INNER JOIN [Dimension].[Stock Item] ON [Fact].[Sale].[Stock Item Key] = [Dimension].[Stock Item].[Stock Item Key]
    INNER JOIN [Dimension].[City] ON [Fact].[Sale].[City Key] = [Dimension].[City].[City Key]
WHERE 
    [Fact].[Sale].[Invoice Date Key] >= DATEADD(qq, DATEDIFF(qq, 0, GETDATE()) - 1, 0)
ORDER BY 
    [Fact].[Sale].[Invoice Date Key] DESC



In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

Ref: https://python.langchain.com/docs/modules/agents/

In [8]:
chat_model.invoke("Echo `Hello World`").content

'Hello World'

In [19]:
from langchain.agents import tool


@tool
def get_word_length(text: str)-> int:
    """Retrun the length of the text"""
    return len(text)


@tool
def exectue_sql_query(sql_query: str)-> str:    
    """Execute SQL query and get the result as stringfy JSON"""
    import json
    import pandas as pd
    from sqldbconn import get_connx

    df = pd.read_sql_query(sql_query, get_connx())
    return json.dumps(df.to_dict(orient='records'))
 

tools = [get_word_length, exectue_sql_query]

In [20]:
from langchain.tools.render import format_tool_to_openai_function


for tool in tools:
    print(format_tool_to_openai_function(tool))

{'name': 'get_word_length', 'description': 'get_word_length(text: str) -> int - Retrun the length of the text', 'parameters': {'title': 'get_word_lengthSchemaSchema', 'type': 'object', 'properties': {'text': {'title': 'Text', 'type': 'string'}}, 'required': ['text']}}
{'name': 'exectue_sql_query', 'description': 'exectue_sql_query(sql_query: str) -> str - Execute SQL query and get the result as stringfy JSON', 'parameters': {'title': 'exectue_sql_querySchemaSchema', 'type': 'object', 'properties': {'sql_query': {'title': 'Sql Query', 'type': 'string'}}, 'required': ['sql_query']}}


```python 
chat_prompt.format(database_schema=db_schema, business_question="Get latest quater sales")
```

In [21]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", user_prompt),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [22]:
chat_model_with_tools = chat_model.bind(functions=[format_tool_to_openai_function(t) for t in tools])

```python
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", user_prompt)
])

msg = chat_prompt.format(database_schema=db_schema, business_question="Get latest quater sales")

```

In [23]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


agent = (
    {
        "database_schema": lambda x: x["database_schema"],
        "business_question": lambda x: x["business_question"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | agent_chat_prompt
    | chat_model_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [24]:
agent

{
  database_schema: RunnableLambda(...),
  business_question: RunnableLambda(...),
  agent_scratchpad: RunnableLambda(...)
}
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'business_question', 'database_schema'], input_types={'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['database_schema'], template='\nYou are a data analyst working for Wide World Importers. \nYou are tasked with writing a SQL query to get latest quater sales from the database. \n\n--- Database Schema ---\n{database_schema}\n--- End of Database Schema ---\n\n## Example \nQuestion: Get top 100 sale records\n\nSQL Query:\n```\nSELECT TOP (100)\nFROM \n    [Fact].[Sale]\n```\

In [25]:
agent.invoke(
    {
        "database_schema": db_schema, 
        "business_question": "Get latest quater sales", 
        "intermediate_steps":[]
    }
)


InvalidRequestError: Unrecognized request argument supplied: functions